<a href="https://colab.research.google.com/github/alan-medina-gomez-lic/Evidencia1_Ago2025/blob/main/evidencia_3_entegable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import os
import sqlite3
from openpyxl.styles import Font, Border, Side, Alignment
from openpyxl import Workbook, utils
import matplotlib.pyplot as plt

TURNOS_POSIBLES = ['MATUTINO', 'VESPERTINO', 'NOCTURNO']
DB_NAME = "reservas_coworking.db"

FECHA_FORMATO_DISPLAY = 'MM/DD/AAAA'
FECHA_FORMATO_INPUT = '%m/%d/%Y'
FECHA_FORMATO_ALMACENAMIENTO = '%Y-%m-%d'

class SistemaReservasSQLite:
    def __init__(self):
        self.conn = sqlite3.connect(DB_NAME)
        self.cursor = self.conn.cursor()
        self.inicializar_db()

    def inicializar_db(self):
        print(f"Buscando estado anterior en la base de datos: {DB_NAME}...")

        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS Clientes (
                Cliente_ID INTEGER PRIMARY KEY,
                Nombre TEXT NOT NULL,
                Apellidos TEXT NOT NULL
            )
        """)

        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS Salas (
                Sala_ID INTEGER PRIMARY KEY,
                Nombre TEXT NOT NULL,
                Cupo INTEGER NOT NULL
            )
        """)

        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS Reservaciones (
                Folio INTEGER PRIMARY KEY,
                Cliente_ID INTEGER NOT NULL,
                Sala_ID INTEGER NOT NULL,
                Fecha TEXT NOT NULL,
                Turno TEXT NOT NULL,
                Evento TEXT NOT NULL,
                FOREIGN KEY(Cliente_ID) REFERENCES Clientes(Cliente_ID),
                FOREIGN KEY(Sala_ID) REFERENCES Salas(Sala_ID),
                UNIQUE(Sala_ID, Fecha, Turno)
            )
        """)
        self.conn.commit()
        print("Estructura de la base de datos lista.")

        self.cursor.execute("SELECT MAX(Folio) FROM Reservaciones")
        max_folio = self.cursor.fetchone()[0]
        self.contador_folios = (max_folio or 0) + 1

        if os.path.exists(DB_NAME) and os.path.getsize(DB_NAME) > 1024:
            print("Estado anterior de la solución recuperado con éxito.")
        else:
            print("No se encontró estado anterior. Se inicia con un estado inicial vacío.")

    def guardar_estado(self):
        try:
            self.conn.commit()
            self.conn.close()
            print("Estado de la solución almacenado (persistencia en SQLite) exitosamente.")
        except Exception as e:
            print(f"Error al cerrar la conexión/guardar el estado: {e}")

    def normalizar_fecha_str(self, fecha_str):
        if not fecha_str:
            return None
        fecha_normalizada = fecha_str.strip().replace('-', '/').replace(' ', '/')
        while '//' in fecha_normalizada:
            fecha_normalizada = fecha_normalizada.replace('//', '/')
        fecha_normalizada = fecha_normalizada.strip('/')
        return fecha_normalizada

    def registrar_sala(self, nombre, cupo):
        try:
            cupo_int = int(cupo)
            if cupo_int <= 0:
                print("Error: El cupo debe ser un número entero positivo.")
                return False
        except ValueError:
            print("Error: El cupo debe ser un número válido.")
            return False

        nombre_limpio = nombre.strip().upper()

        self.cursor.execute("SELECT Sala_ID FROM Salas WHERE Nombre = ?", (nombre_limpio,))
        if self.cursor.fetchone():
            print(f"Error: La sala '{nombre_limpio}' ya está registrada.")
            return False

        self.cursor.execute(
            "INSERT INTO Salas (Nombre, Cupo) VALUES (?, ?)",
            (nombre_limpio, cupo_int)
        )
        sala_clave = self.cursor.lastrowid
        self.conn.commit()

        print(f"Sala '{nombre_limpio}' registrada con éxito. Clave: {sala_clave}, Cupo: {cupo_int}.")
        return True

    def registrar_cliente(self, nombre, apellidos):
        nombre_limpio = nombre.strip()
        apellidos_limpios = apellidos.strip()

        self.cursor.execute(
            "SELECT Cliente_ID FROM Clientes WHERE Nombre = ? AND Apellidos = ?",
            (nombre_limpio, apellidos_limpios)
        )
        if self.cursor.fetchone():
            print(f"Error: El cliente '{nombre_limpio} {apellidos_limpios}' ya está registrado.")
            return False

        self.cursor.execute(
            "INSERT INTO Clientes (Nombre, Apellidos) VALUES (?, ?)",
            (nombre_limpio, apellidos_limpios)
        )
        cliente_clave = self.cursor.lastrowid
        self.conn.commit()

        print(f"Cliente '{nombre_limpio} {apellidos_limpios}' registrado con éxito. Clave: C{cliente_clave}.")
        return True

    def validar_fecha(self, fecha_str):

        fecha_normalizada = self.normalizar_fecha_str(fecha_str)
        if not fecha_normalizada:
            return None

        try:
            fecha_reserva = datetime.strptime(fecha_normalizada, FECHA_FORMATO_INPUT).date()
        except ValueError:
            print(f"Error: Formato de fecha incorrecto. Use el formato {FECHA_FORMATO_DISPLAY} (ej. 10/29/2025).")
            return None

        fecha_actual_sistema = datetime.now().date()
        fecha_minima_reserva = fecha_actual_sistema + timedelta(days=2)

        if fecha_reserva < fecha_minima_reserva:
            print(f"Error: La fecha debe ser al menos dos días posteriores a hoy. (Mínimo: {fecha_minima_reserva.strftime(FECHA_FORMATO_INPUT)}).")
            return None

        if fecha_reserva.weekday() == 6:
            print("Error: No se permiten reservaciones para domingos.")
            fecha_propuesta = fecha_reserva + timedelta(days=1)

            while True:
                resp = input(f"¿Desea reservar el lunes {fecha_propuesta.strftime(FECHA_FORMATO_INPUT)}? (S/N): ").strip().upper()
                if resp == 'S':
                    return fecha_propuesta
                elif resp == 'N':
                    return None
                else:
                    print("Respuesta inválida.")

        return fecha_reserva

    def seleccionar_cliente(self):
        self.cursor.execute("SELECT Cliente_ID, Nombre, Apellidos FROM Clientes ORDER BY Apellidos, Nombre")
        clientes_db = self.cursor.fetchall()

        if not clientes_db:
            print("No hay clientes registrados. Debe registrar un cliente primero.")
            return None

        while True:
            print("\n--- CLIENTES REGISTRADOS (Clave: Apellidos, Nombre) ---")
            for id, nombre, apellidos in clientes_db:
                print(f"[C{id}]: {apellidos}, {nombre}")
            print("-----------------------------------------------------")

            clave_ingresada = input("Ingrese la CLAVE del cliente (ej. C1) o 'C' para cancelar: ").strip().upper()

            if clave_ingresada == 'C':
                print("Operación de reserva cancelada.")
                return None

            if not clave_ingresada.startswith('C') or not clave_ingresada[1:].isdigit():
                print("Error: La clave debe iniciar con 'C' seguido de un número. Intente de nuevo.")
                continue

            cliente_id_int = int(clave_ingresada[1:])

            if any(id == cliente_id_int for id, _, _ in clientes_db):
                return cliente_id_int
            else:
                print(f"La clave '{clave_ingresada}' no existe. Intente de nuevo.")

    def registrar_reservacion(self):
        print("\n\n*** REGISTRAR NUEVA RESERVACIÓN ***")

        self.cursor.execute("SELECT COUNT(*) FROM Clientes")
        if self.cursor.fetchone()[0] == 0:
            print("Error: No hay clientes registrados.")
            return

        self.cursor.execute("SELECT COUNT(*) FROM Salas")
        if self.cursor.fetchone()[0] == 0:
            print("Error: No hay salas registradas.")
            return

        cliente_id = self.seleccionar_cliente()
        if cliente_id is None:
            return

        while True:
            fecha_str = input(f"Ingrese la FECHA de la reserva ({FECHA_FORMATO_DISPLAY}): ").strip()
            if not fecha_str: continue

            fecha_reserva = self.validar_fecha(fecha_str)
            if fecha_reserva:
                fecha_almacenar = fecha_reserva.strftime(FECHA_FORMATO_ALMACENAMIENTO)
                break

        self.cursor.execute(
            "SELECT Sala_ID, Turno FROM Reservaciones WHERE Fecha = ?",
            (fecha_almacenar,)
        )
        slots_ocupados = set(self.cursor.fetchall())

        self.cursor.execute("SELECT Sala_ID, Nombre, Cupo FROM Salas")
        salas_db = self.cursor.fetchall()

        slots_disponibles = []

        print(f"\n--- DISPONIBILIDAD PARA EL DÍA {fecha_reserva.strftime(FECHA_FORMATO_INPUT)} ---")
        print("ID | NOMBRE | CUPO | TURNO(S) DISPONIBLE(S)")
        print("-------------------------------------------------")

        for sala_id, nombre, cupo in salas_db:
            turnos_disponibles_sala = []
            for turno in TURNOS_POSIBLES:
                slot = (sala_id, turno)
                if slot not in slots_ocupados:
                    turnos_disponibles_sala.append(turno)
                    slots_disponibles.append(slot)

            if turnos_disponibles_sala:
                print(f"{sala_id:<2} | {nombre:<6} | {cupo:<4} | {', '.join(turnos_disponibles_sala)}")
            else:
                print(f"{sala_id:<2} | {nombre:<6} | {cupo:<4} | (COMPLETO)")

        if not slots_disponibles:
            print("\nLo sentimos, no hay slots (sala/turno) disponibles para esa fecha.")
            return

        self.cursor.execute(
            "SELECT Turno FROM Reservaciones WHERE Fecha = ? AND Cliente_ID = ?",
            (fecha_almacenar, cliente_id)
        )
        turnos_reservados_por_el_cliente = [row[0] for row in self.cursor.fetchall()]
        turnos_disponibles_para_cliente = [t for t in TURNOS_POSIBLES if t not in turnos_reservados_por_el_cliente]

        turnos_prompt_texto = ", ".join(turnos_disponibles_para_cliente)
        slot_elegido = None

        while slot_elegido is None:
            sala_elegida_id = None

            while sala_elegida_id is None:
                entrada_sala = input("\nIngrese la ID de la sala a reservar (o 'C' para cancelar): ").strip().upper()

                if entrada_sala == 'C':
                    print("Operación de reserva de sala/turno cancelada.")
                    return

                if not entrada_sala.isdigit():
                    print("Error: La ID de sala debe ser un número (ej. 1). Intente de nuevo.")
                    continue

                sala_id_int = int(entrada_sala)

                if any(id == sala_id_int for id, _, _ in salas_db):
                    sala_elegida_id = sala_id_int
                else:
                    print(f"Error: La ID de sala '{sala_id_int}' no existe. Intente de nuevo.")

            turno_elegido = None
            while turno_elegido is None:
                turno_input = input(f"Ingrese el TURNO ({turnos_prompt_texto}): ").strip().upper()

                if turno_input in turnos_disponibles_para_cliente:
                    slot_intento = (sala_elegida_id, turno_input)
                    if slot_intento in slots_disponibles:
                        slot_elegido = slot_intento
                        turno_elegido = turno_input
                    else:
                        print("Combinación de Sala/Turno no disponible. Revise el listado.")
                        break
                else:
                    if turno_input in TURNOS_POSIBLES:
                         print(f"Error: El turno '{turno_input}' no está disponible para esa sala, o ya lo tiene reservado usted.")
                    else:
                         print(f"Error: El turno '{turno_input}' no es válido. Los turnos posibles son: {turnos_prompt_texto}.")

            if slot_elegido is None and entrada_sala != 'C':
                continue

        sala_id_final, turno_final = slot_elegido

        while True:
            nombre_evento = input("Ingrese el NOMBRE del evento: ").strip().upper()
            if nombre_evento:
                break
            print("El nombre del evento no puede estar vacío.")

        self.cursor.execute(
            "INSERT INTO Reservaciones (Cliente_ID, Sala_ID, Fecha, Turno, Evento) VALUES (?, ?, ?, ?, ?)",
            (cliente_id, sala_id_final, fecha_almacenar, turno_final, nombre_evento))
        folio_unico = self.cursor.lastrowid
        self.conn.commit()

        print(f"\nRESERVACIÓN REGISTRADA CON ÉXITO. FOLIO: F{folio_unico}")
        self.contador_folios = folio_unico + 1


    def editar_reservacion(self):
        print("\n\n*** EDITAR NOMBRE DE EVENTO ***")

        self.cursor.execute("SELECT COUNT(*) FROM Reservaciones")
        if self.cursor.fetchone()[0] == 0:
            print("No hay reservaciones registradas para editar.")
            return

        while True:
            fecha_inicio_str = input(f"Ingrese FECHA DE INICIO del rango ({FECHA_FORMATO_DISPLAY}): ").strip()
            fecha_fin_str = input(f"Ingrese FECHA DE FIN del rango ({FECHA_FORMATO_DISPLAY}): ").strip()

            fecha_inicio_normalizada = self.normalizar_fecha_str(fecha_inicio_str)
            fecha_fin_normalizada = self.normalizar_fecha_str(fecha_fin_str)

            try:
                fecha_inicio = datetime.strptime(fecha_inicio_normalizada, FECHA_FORMATO_INPUT).strftime(FECHA_FORMATO_ALMACENAMIENTO)
                fecha_fin = datetime.strptime(fecha_fin_normalizada, FECHA_FORMATO_INPUT).strftime(FECHA_FORMATO_ALMACENAMIENTO)
                if fecha_inicio <= fecha_fin:
                    break
                else:
                    print("La fecha de inicio debe ser anterior o igual a la fecha de fin.")
            except ValueError:
                print(f"Error: Formato de fecha incorrecto. Use {FECHA_FORMATO_DISPLAY}.")

        self.cursor.execute(
            """
            SELECT Folio, Evento, Fecha
            FROM Reservaciones
            WHERE Fecha BETWEEN ? AND ?
            ORDER BY Fecha
            """,
            (fecha_inicio, fecha_fin))
        eventos_en_rango = self.cursor.fetchall()

        if not eventos_en_rango:
            print(f"No se encontraron eventos entre {fecha_inicio_str} y {fecha_fin_str}.")
            return

        while True:
            print("\n--- EVENTOS ENCONTRADOS ---")
            print(f"{'FOLIO':<7} {'FECHA':<15} {'EVENTO'}")
            print("----------------------------------")
            for folio, evento, fecha in eventos_en_rango:
                fecha_display = datetime.strptime(fecha, FECHA_FORMATO_ALMACENAMIENTO).strftime(FECHA_FORMATO_INPUT)
                print(f"F{folio:<6} {fecha_display:<15} {evento}")
            print("----------------------------")

            folio_a_modificar_str = input("Ingrese el FOLIO (ej. F1) del evento a modificar o 'C' para cancelar: ").strip().upper()

            if folio_a_modificar_str == 'C':
                print("Edición cancelada.")
                return

            if not folio_a_modificar_str.startswith('F') or not folio_a_modificar_str[1:].isdigit():
                print("Error: El folio debe iniciar con 'F' seguido de un número. Intente de nuevo.")
                continue

            folio_a_modificar_int = int(folio_a_modificar_str[1:])

            if any(f[0] == folio_a_modificar_int for f in eventos_en_rango):
                while True:
                    nuevo_evento = input(f"Ingrese el nuevo nombre para el evento 'F{folio_a_modificar_int}': ").strip().upper()
                    if nuevo_evento:
                        break
                    print("El nombre del evento no puede omitirse.")

                self.cursor.execute(
                    "UPDATE Reservaciones SET Evento = ? WHERE Folio = ?",
                    (nuevo_evento, folio_a_modificar_int)
                )
                self.conn.commit()
                print(f"Evento con FOLIO F{folio_a_modificar_int} modificado a: '{nuevo_evento}'.")
                return
            else:
                print(f"Folio '{folio_a_modificar_str}' no encontrado en el rango de fechas. Intente de nuevo.")


    def consultar_reservaciones(self):
        print("\n\n*** CONSULTAR RESERVACIONES ***")

        while True:
            fecha_str = input(f"Ingrese la FECHA a consultar ({FECHA_FORMATO_DISPLAY}) o deje vacío para hoy: ").strip()

            if not fecha_str:
                fecha_consulta_dt = datetime.now().date()
                print(f"Asumiendo fecha actual: {fecha_consulta_dt.strftime(FECHA_FORMATO_INPUT)}")
                break

            fecha_normalizada = self.normalizar_fecha_str(fecha_str)
            if not fecha_normalizada:
                continue

            try:
                fecha_consulta_dt = datetime.strptime(fecha_normalizada, FECHA_FORMATO_INPUT).date()
                break
            except ValueError:
                print(f"Error: Formato de fecha incorrecto. Use {FECHA_FORMATO_DISPLAY}.")

        fecha_almacenamiento = fecha_consulta_dt.strftime(FECHA_FORMATO_ALMACENAMIENTO)
        fecha_display = fecha_consulta_dt.strftime(FECHA_FORMATO_INPUT)

        self.cursor.execute(
            """
            SELECT
                S.Nombre, C.Apellidos, C.Nombre, R.Evento, R.Turno
            FROM Reservaciones R
            JOIN Salas S ON R.Sala_ID = S.Sala_ID
            JOIN Clientes C ON R.Cliente_ID = C.Cliente_ID
            WHERE R.Fecha = ?
            ORDER BY S.Nombre, R.Turno
            """,
            (fecha_almacenamiento,)
        )
        reservas_del_dia = self.cursor.fetchall()

        if not reservas_del_dia:
            print(f"No hay reservaciones para la fecha {fecha_display}.")
            return

        data = []
        for sala_nombre, apell, nombre, evento, turno in reservas_del_dia:
             cliente_nombre_completo = f"{apell}, {nombre}"
             data.append([sala_nombre, cliente_nombre_completo, evento, turno])

        reporte_df = pd.DataFrame(data, columns=['SALA', 'CLIENTE', 'EVENTO', 'TURNO'])

        ANCHO_TOTAL = 100
        print("\n" + "*" * ANCHO_TOTAL)
        texto_encabezado = f"REPORTE DE RESERVACIONES PARA EL DÍA {fecha_display}"
        print(f"** {texto_encabezado:^94} **")
        print("*" * ANCHO_TOTAL)

        print(f"{'SALA':<15} {'CLIENTE':<30} {'EVENTO':<35} {'TURNO':<15}")
        print("*" * ANCHO_TOTAL)

        for _, row in reporte_df.iterrows():
            sala = str(row['SALA'])[:14]
            cliente = str(row['CLIENTE'])[:29]
            evento = str(row['EVENTO'])[:34]
            turno = str(row['TURNO'])[:14]

            print(f"{sala:<15} {cliente:<30} {evento:<35} {turno:<15}")

        print("*" * 17 + " FIN DEL REPORTE " + "*" * (ANCHO_TOTAL - 17 - 17 - 3) + "\n")

        self.submenu_exportacion(fecha_almacenamiento, reporte_df)


    def submenu_exportacion(self, fecha_almacenamiento, reporte_df):
        while True:
            exportar = input("\n¿Desea exportar el reporte? (C=CSV, J=JSON, E=Excel, N=No): ").strip().upper()
            if exportar == 'C':
                self._exportar_a_csv(fecha_almacenamiento, reporte_df)
                break
            elif exportar == 'J':
                self._exportar_a_json(fecha_almacenamiento, reporte_df)
                break
            elif exportar == 'E':
                self._exportar_a_excel(fecha_almacenamiento, reporte_df)
                break
            elif exportar == 'N':
                break
            else:
                print("Opción inválida. Intente de nuevo.")

    def _exportar_a_csv(self, fecha_consulta_str, reporte_df):
        archivo_nombre = f"Reporte_Reservas_{fecha_consulta_str}.csv"
        try:
            reporte_df.to_csv(archivo_nombre, index=False, encoding='utf-8')
            print(f"Reporte exportado exitosamente a CSV: {archivo_nombre}")
        except Exception as e:
            print(f"Error al guardar el archivo CSV: {e}")

    def _exportar_a_json(self, fecha_consulta_str, reporte_df):
        archivo_nombre = f"Reporte_Reservas_{fecha_consulta_str}.json"
        try:
            reporte_df.to_json(archivo_nombre, orient='records', indent=4)
            print(f"Reporte exportado exitosamente a JSON: {archivo_nombre}")
        except Exception as e:
            print(f"Error al guardar el archivo JSON: {e}")

    def _exportar_a_excel(self, fecha_consulta_str, reporte_df):
        archivo_nombre = f"Reporte_Reservas_{fecha_consulta_str}.xlsx"

        wb = Workbook()
        ws = wb.active
        headers = reporte_df.columns.tolist()

        font_bold = Font(bold=True)
        thin_border = Side(border_style="thin", color="000000")
        thick_border = Side(border_style="thick", color="000000")
        border_bottom_thick = Border(top=thin_border, left=thin_border, right=thin_border, bottom=thick_border)
        center_alignment = Alignment(horizontal='center', vertical='center')

        fecha_display = datetime.strptime(fecha_consulta_str, FECHA_FORMATO_ALMACENAMIENTO).strftime(FECHA_FORMATO_INPUT)

        titulo = f"REPORTE DE RESERVACIONES PARA EL DÍA {fecha_display}"
        ws['A1'] = titulo
        ws['A1'].font = font_bold
        ws.merge_cells(start_row=1, start_column=1, end_row=1, end_column=len(headers))
        ws['A1'].alignment = center_alignment

        ws.append(headers)
        header_row_num = ws.max_row

        for col_idx, header_text in enumerate(headers):
            col_letter = utils.get_column_letter(col_idx + 1)
            header_cell = ws[f'{col_letter}{header_row_num}']

            header_cell.font = font_bold
            header_cell.border = border_bottom_thick
            header_cell.alignment = center_alignment
            ws.column_dimensions[col_letter].width = max(len(header_text), 15) * 1.5

        for row_data in reporte_df.values.tolist():
            ws.append(row_data)
            data_row_num = ws.max_row

            for cell in ws[data_row_num]:
                cell.alignment = center_alignment

        try:
            wb.save(archivo_nombre)
            print(f"Reporte exportado exitosamente a Excel: {archivo_nombre}")
        except Exception as e:
            print(f"Error al guardar el archivo Excel: {e}")


def menu_principal():
    if os.path.exists(DB_NAME):
        print("\n" + "=" * 40)
        print(f"ATENCIÓN: Se encontró la base de datos '{DB_NAME}'.")
        reset = input("¿Desea INICIAR DESDE CERO (eliminar DB)? (S/N): ").strip().upper()
        if reset == 'S':
            try:
                os.remove(DB_NAME)
                print("Base de datos anterior eliminada. Iniciando sistema vacío.")
            except Exception as e:
                print(f"Error al eliminar la DB: {e}")
        print("=" * 40)

    sistema = SistemaReservasSQLite()

    while True:
        print("\n" + "=" * 40)
        print("      SISTEMA DE RESERVAS COWORKING (SQLite)")
        print("=" * 40)
        print("[1] Registrar la reservación de una sala")
        print("[2] Editar el nombre del evento")
        print("[3] Consultar las reservaciones existentes y Exportar")
        print("[4] Registrar a un nuevo cliente")
        print("[5] Registrar una sala")
        print("[6] Salir (Guardar Estado)")
        print("-" * 40)

        opcion = input("Seleccione una opción: ").strip()

        if opcion == '1':
            sistema.registrar_reservacion()
        elif opcion == '2':
            sistema.editar_reservacion()
        elif opcion == '3':
            sistema.consultar_reservaciones()
        elif opcion == '4':
            print("\n*** REGISTRAR NUEVO CLIENTE ***")
            while True:
                nombre = input("Nombre del cliente: ")
                if not nombre.strip():
                    print("Error: El nombre del cliente no puede estar vacío.")
                else: break
            while True:
                apellidos = input("Apellidos del cliente: ")
                if not apellidos.strip():
                    print("Error: Los apellidos del cliente no pueden estar vacíos.")
                else: break
            sistema.registrar_cliente(nombre, apellidos)

        elif opcion == '5':
            print("\n*** REGISTRAR NUEVA SALA ***")
            while True:
                nombre = input("Nombre de la sala: ")
                if not nombre.strip():
                    print("Error: El nombre de la sala no puede estar vacío.")
                else: break
            while True:
                cupo = input("Cupo de la sala (entero > 0): ")
                if not cupo.strip() or not cupo.isdigit() or int(cupo) <= 0:
                    print("Error: El cupo debe ser un número entero mayor a cero.")
                else: break
            sistema.registrar_sala(nombre, cupo)

        elif opcion == '6':
            confirmacion = input("¿Está seguro que desea salir? (S/N): ").upper()
            if confirmacion == 'S':
                sistema.guardar_estado()
                print("Saliendo del sistema. ¡Adiós!")
                break
            else:
                print("Volviendo al menú principal.")

        else:
            print("Opción no válida. Intente de nuevo.")


if __name__ == "__main__":
    menu_principal()